#### Initial Setup

In [ ]:
#imported libraries
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import boto
import numpy as np
from numpy import matlib
import scipy as sp
import math
import matplotlib.pyplot as plt
import langdetect
import datetime
%matplotlib inline  
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import Imputer

import os
import findspark; findspark.init()
import pyspark
from pyspark.sql.types import *
from pyspark.sql.types import Row
import pyspark.sql.functions as func
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils

from pyspark.mllib.linalg import Vectors 
from pyspark.mllib.regression import LabeledPoint  
from pyspark.mllib.regression import LinearRegressionWithSGD 

os.environ["PYSPARK_SUBMIT_ARGS"] = (
  "--packages com.databricks:spark-csv_2.11:1.4.0 pyspark-shell"
)

#NEED TO ADD "  SPARK_DRIVER_MEMORY=5G   "  to ./conf/spark-env.sh 

try:
    sc = pyspark.SparkContext()
except Exception as e:
    print "SparkContext exists... Continuing on."
    
sqlCtx = pyspark.sql.SQLContext(sc)
sc.setCheckpointDir('checkpoint/')

#### Load files

In [ ]:
sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID'])
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY'])

In [ ]:
downloads = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/train_app_downloads.csv').drop('')
ratings = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/train_app_rating.csv').drop('')
usages = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/train_usage.csv').drop('')
revenues = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/train_revenue.csv').drop('')
output = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/train_final_downloads.csv').drop('')
prev_downloads = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/train_cumulative_downloads_2015-02.csv').drop('')  
release_date = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/train_release_date.csv').drop('')
text_score = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='false',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/sentiment.csv').drop('')
title_score = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='false',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/t_sentiment.csv').drop('')
avg_score = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/avg_sent_score.csv').drop('')

reviews_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("country",StringType(),True),
    StructField("rating",IntegerType(),True),
    StructField("date",StringType(),True),
    StructField("title",StringType(),True),
    StructField("version",StringType(),True),
    StructField("text",StringType(),True),
    StructField("reviewer",StringType(),True)
])
reviews = pd.read_csv('s3://cs341bucket1/Data/train_app_review.csv')
reviews = sqlCtx.createDataFrame(reviews,reviews_schema)

In [ ]:
#usage imputation
imputer = Imputer(missing_values=-1, strategy='median', axis=0)
pd_usages = usages.toPandas()
category = list(set(pd_usages["category"].values))
imp = pd.DataFrame(columns = pd_usages.columns)
for cat in category:
    #for dev in ["iphone","ipad"]:
        for metric in range(1,5):
            curr_df = pd_usages.ix[pd_usages["category"]==cat,:]
            #curr_df = curr_df.ix[curr_df["device"]==dev,:]
            curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            name = curr_df.columns
            df1 = curr_df.ix[:,0:6]
            df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,6:]))
            df2.index = df1.index
            curr_df = pd.concat([df1,df2],axis = 1)
            curr_df.columns = name 
            imp = pd.concat([imp,curr_df],axis = 0)
usages = sqlCtx.createDataFrame(imp)

#revenue imputation
pd_revenues = revenues.toPandas()
imp = pd.DataFrame(columns = revenues.columns)
for cat in category:
    for dev in ["iphone","ipad"]:
        #for metric in range(1,5):
            curr_df = pd_revenues.ix[pd_revenues["category"]==cat,:]
            curr_df = curr_df.ix[curr_df["device"]==dev,:]
            #curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            name = curr_df.columns
            df1 = curr_df.ix[:,0:5]
            df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,5:]))
            df2.index = df1.index
            curr_df = pd.concat([df1,df2],axis = 1)
            curr_df.columns = name 
            imp = pd.concat([imp,curr_df],axis = 0)
revenues = sqlCtx.createDataFrame(imp)



#### Generate Predictors

In [ ]:
old_dateRange = pd.date_range('03/01/2015', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))
dateRange = pd.date_range('03/01/2015', periods=56).format(formatter=lambda x: x.strftime('%m_%d_%Y'))
for d in range(56):
    revenues = revenues.withColumnRenamed(old_dateRange[d],dateRange[d])
    usages = usages.withColumnRenamed(old_dateRange[d],dateRange[d])
    downloads = downloads.withColumnRenamed(old_dateRange[d],dateRange[d])  
output = output.withColumnRenamed("cumulative_downloads_2016-02","cumulative_downloads_2016_02")
prev_downloads = prev_downloads.withColumnRenamed("cumulative_downloads_2015-02","cumulative_downloads_2015_02")

In [ ]:
#Initialization
predictors = downloads['id','name','category','device']

In [ ]:
# Generate the weekly downloads

sqlCtx.registerDataFrameAsTable(downloads, "downloads")
sqlCtx.registerDataFrameAsTable(usages, "usages")
sqlCtx.registerDataFrameAsTable(revenues, "revenues")

def get_log_week(*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    nb_0 = args.count(0)
    if nb_0 == 7:
        return 0
    return math.log(1.0*sum(args)/(7-nb_0))
sqlCtx.registerFunction("get_log_week", get_log_week,returnType=FloatType())
def get_download_sum(*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    return (1.0*sum(args))
sqlCtx.registerFunction("get_download_sum", get_download_sum,returnType=FloatType())

predictors = sqlCtx.sql("SELECT id, name, category, device, \
           get_log_week("+ ",".join(dateRange[0:7])+") AS week_1 \
            ,get_log_week("+",".join(dateRange[7:14])+") AS week_2 \
            ,get_log_week("+ ",".join(dateRange[14:21])+") AS week_3 \
           ,get_log_week("+",".join(dateRange[21:28])+") AS week_4 \
           ,get_log_week("+",".join(dateRange[28:35])+") AS week_5 \
           ,get_log_week("+",".join(dateRange[35:42])+") AS week_6 \
           ,get_log_week("+",".join(dateRange[42:49])+") AS week_7 \
           ,get_log_week("+",".join(dateRange[49:56])+") AS week_8\
           ,get_download_sum("+ ",".join(dateRange)+") AS download_sum \
           from downloads")
sqlCtx.registerDataFrameAsTable(predictors, "predictors")

#I workaround the error by this modification. 
#I don't know why I couldn't run the code before but this workaround gives the same result.
#predictors = sqlCtx.sql("SELECT "+', '.join(predictors.columns)+", week_1+week_2+week_3+week_4+week_5+week_6+week_7+week_8 AS download_sum FROM predictors")


In [ ]:
m1 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 1")
m2 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 2")
m3 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 3")
m4 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 4")
sqlCtx.registerDataFrameAsTable(m1,"m1")
sqlCtx.registerDataFrameAsTable(m2,"m2")
sqlCtx.registerDataFrameAsTable(m3,"m3")
sqlCtx.registerDataFrameAsTable(m4,"m4")
sqlCtx.registerDataFrameAsTable(avg_score,"avg_score")

In [ ]:
# Make coefficients

def get_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = [x if not (x==-1) else 0 for x in list(args)]
    return  float(np.polyfit(range(56),np.cumsum(args[1:]),args[0])[0])
    
#Generate the step max and min 
def get_maxStep(maximum,*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    if (np.count_nonzero(args) == 0):
        return 0
    m = 0
    for d in range(1,56):
        if (args[d]!=0 and args[d-1]!=0):
            c = (args[d]-args[d-1])
            if (maximum and m < c):
                m = c
            if ( not maximum and m > c):
                m = c
    return m

def get_std(*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    return float(np.std(list(args)))

def get_nbMissing(*args):
    return list(args).count(-1)

#Generate the daily average
def get_dailyAvg(*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    if (np.count_nonzero(args) == 0):
        return 0
    return  (1.0*sum(args)/np.count_nonzero(args))

def get_usage_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    if -1 in args: return 0
    return  float(np.polyfit(range(8),args[1:],args[0])[0])

def get_revenue_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    if -1 in args: return 0
    return  float(np.polyfit(range(56),args[1:],args[0])[0])

def get_usage_max(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    args = [c for c in args[1:] if c!=-1]
    if len(args)==0: return 0
    return  float(np.max(args))

def get_usage_mean(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    args = [c for c in args[1:] if c!=-1]
    if len(args)==0: return 0
    return  float(np.mean(args))

def get_revenue_max(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    args = [c for c in args[1:] if c!=-1]
    if len(args)==0: return 0
    return  float(np.max(args))

def get_revenue_mean(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    args = [c for c in args[1:] if c!=-1]
    if len(args)==0: return 0
    return  float(np.mean(args))

sqlCtx.registerFunction("get_nbMissing", get_nbMissing,returnType=IntegerType())
sqlCtx.registerFunction("get_std", get_std,returnType=FloatType())
sqlCtx.registerFunction("get_maxStep", get_maxStep,returnType=IntegerType())
sqlCtx.registerFunction("get_coefficients", get_coefficients,returnType=FloatType())
sqlCtx.registerFunction("daily_avg", get_dailyAvg,returnType=FloatType())
sqlCtx.registerFunction("get_usage_coefficients", get_usage_coefficients,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_coefficients", get_revenue_coefficients,returnType=FloatType())
sqlCtx.registerFunction("get_usage_max", get_usage_max,returnType=FloatType())
sqlCtx.registerFunction("get_usage_mean", get_usage_mean,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_max", get_revenue_max,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_mean", get_revenue_mean,returnType=FloatType())

temp_downloads = sqlCtx.sql("SELECT id,name,category, device \
, get_coefficients(0,"+",".join(dateRange)+") AS coef_0 \
,get_coefficients(1,"+",".join(dateRange)+") AS coef_1 \
,get_coefficients(2,"+",".join(dateRange)+") AS coef_2 \
,get_coefficients(3,"+",".join(dateRange)+") AS coef_3 \
,get_maxStep(True,"+",".join(dateRange)+") AS max_step \
,get_maxStep(False,"+",".join(dateRange)+") AS min_step \
,get_std("+",".join(dateRange)+") AS downloads_std \
,get_nbMissing("+",".join(dateRange)+") AS nb_missing \
,daily_avg(" + ",".join(dateRange[0:56]) + ") AS daily_avg \
 FROM downloads")

temp_m1 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+",".join(m1.columns[5:13])+") AS m1_coef_0, \
get_usage_coefficients(1,"+",".join(m1.columns[5:13])+") AS m1_coef_1, \
get_usage_coefficients(2,"+",".join(m1.columns[5:13])+") AS m1_coef_2, \
get_usage_max(0,"+",".join(m1.columns[5:13])+") AS m1_max, \
get_usage_mean(0,"+",".join(m1.columns[5:13])+") AS m1_mean FROM m1")

temp_m2 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+",".join(m2.columns[5:13])+") AS m2_coef_0, \
get_usage_coefficients(1,"+",".join(m2.columns[5:13])+") AS m2_coef_1, \
get_usage_coefficients(2,"+",".join(m2.columns[5:13])+") AS m2_coef_2, \
get_usage_max(0,"+",".join(m2.columns[5:13])+") AS m2_max, \
get_usage_mean(0,"+",".join(m2.columns[5:13])+") AS m2_mean FROM m2")

temp_m3 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+",".join(m3.columns[5:13])+") AS m3_coef_0, \
get_usage_coefficients(1,"+",".join(m3.columns[5:13])+") AS m3_coef_1, \
get_usage_coefficients(2,"+",".join(m3.columns[5:13])+") AS m3_coef_2, \
get_usage_max(0,"+",".join(m3.columns[5:13])+") AS m3_max, \
get_usage_mean(0,"+",".join(m3.columns[5:13])+") AS m3_mean FROM m3")

temp_m4 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+",".join(m4.columns[5:13])+") AS m4_coef_0, \
get_usage_coefficients(1,"+",".join(m4.columns[5:13])+") AS m4_coef_1, \
get_usage_coefficients(2,"+",".join(m4.columns[5:13])+") AS m4_coef_2, \
get_usage_max(0,"+",".join(m4.columns[5:13])+") AS m4_max, \
get_usage_mean(0,"+",".join(m4.columns[5:13])+") AS m4_mean FROM m4")

temp_revenues = sqlCtx.sql("SELECT id, name, category, device, \
get_revenue_coefficients(0,"+",".join(revenues.columns[4:])+") AS rev_coef_0, \
get_revenue_coefficients(1,"+",".join(revenues.columns[4:])+") AS rev_coef_1, \
get_revenue_coefficients(2,"+",".join(revenues.columns[4:])+") AS rev_coef_2, \
get_revenue_max(0,"+",".join(revenues.columns[4:])+") AS rev_max, \
get_revenue_mean(0,"+",".join(revenues.columns[4:])+") AS rev_mean FROM revenues")

predictors = predictors.join(temp_downloads,["id","name","category","device"],how='left_outer')
predictors = predictors.join(temp_revenues,["id", "name", "category","device"],how='left_outer')
predictors = predictors.join(temp_m1,["id", "name", "category","device"],how='left_outer')
predictors = predictors.join(temp_m2,["id", "name", "category","device"],how='left_outer')
predictors = predictors.join(temp_m3,["id", "name", "category","device"],how='left_outer')
predictors = predictors.join(temp_m4,["id", "name", "category","device"],how='left_outer')
predictors = predictors.join(avg_score,["id", "name", "category","device"],how='left_outer')
#predictors = predictors.join(temp_usages,["id", "name", "category","device"],how='left_outer')
#predictors = predictors.join(,)



In [ ]:
# previous downloads addition
predictors = predictors.join(prev_downloads,["id","device"],how='left_outer')

In [ ]:
# Days since release generation with imputation
def get_days(date, id):
    if date == "no_date":
        # return 0
        id = 1.0*id/100000000
        return int(5856.25394104 -1731.74798728*id+195.553086*id**2  -8.12861635*id**3)
    else:
        return (datetime.datetime.strptime('03/01/2015', '%m/%d/%Y').date() \
            - datetime.datetime.strptime(date, '%Y-%m-%d').date()).days



release_date = downloads[["id"]].dropDuplicates().join(release_date,["id"],"left").fillna("no_date",["release_date"])
sqlCtx.registerDataFrameAsTable(release_date, "release_date")
sqlCtx.registerFunction("get_days", get_days,returnType=IntegerType())
temp_date = sqlCtx.sql("SELECT id\
, get_days(release_date, id) AS days_since_release \
 FROM release_date")

predictors = predictors.join(temp_date,["id"],"left")

In [ ]:
#ratings generation
sqlCtx.registerDataFrameAsTable(ratings, "ratings")
temp_ratings = sqlCtx.sql("SELECT id,name,category \
, 1.0*start1/(start1+star2+star3+star4+star5) AS star1 \
, 1.0*star2/(start1+star2+star3+star4+star5) AS star2 \
, 1.0*star3/(start1+star2+star3+star4+star5) AS star3 \
, 1.0*star4/(start1+star2+star3+star4+star5) AS star4 \
, 1.0*star5/(start1+star2+star3+star4+star5) AS star5 \
, (start1+star2+star3+star4+star5) AS num_ratings \
 FROM ratings")

predictors = predictors.join(temp_ratings,["id","name","category"],"left")

In [ ]:
# Categories
list_categories = [ x.category.replace(" ","_") for x in sqlCtx.sql("SELECT category \
 FROM downloads\
 group by category \
 ").collect()]
for cat in list_categories:
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, CASE WHEN (category = "'''+cat+'''") THEN 1 ELSE 0 END AS '''+cat+''' FROM predictors''')


In [ ]:
# Device
sqlCtx.registerDataFrameAsTable(predictors, "predictors")
predictors=sqlCtx.sql('''SELECT *, CASE WHEN (device = "iphone") THEN 1 ELSE 0 END AS iphone FROM predictors''')
sqlCtx.registerDataFrameAsTable(predictors, "predictors")
predictors=sqlCtx.sql('''SELECT *, CASE WHEN (device = "ipad") THEN 1 ELSE 0 END AS ipad FROM predictors''')

In [ ]:
lang = ['ja','zh-cn','ko','en','other']
def get_language(x):
    try:
        detected = langdetect.detect_langs(x.decode('utf8','ignore'))[0]
        if detected.prob < 0.7:
            return "other"
        elif  detected.lang in lang:
            return detected.lang
        else:
            return "other"
    except:
        return "other"
sqlCtx.registerFunction("get_language", get_language,returnType=StringType())

In [ ]:
#Language of the title
for l in lang:
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, CASE WHEN (get_language(name) = "'''+l+'''") THEN 1 \
    ELSE 0 END AS '''+l.replace("-","_")+''' FROM predictors''')

In [ ]:
#Reviews 
#escape is used in case some asshole used - or [space] anywhere
def escape(text):
    return text.replace(" ","_").replace("-","_")
# number of reviews
def get_recentReviews(date):
    return int((datetime.datetime.strptime('04/01/2015', '%m/%d/%Y').date() \
            - datetime.datetime.strptime(date, '%Y-%m-%d').date()).days >=0)

#First step
list_countries =['United_States', 'France', 'Japan', 'Spain', 'United_Kingdom','Saudi_Arabia', 'Germany'\
     , 'Hong_Kong', 'Switzerland', 'Turkey','Netherlands', 'Australia', 'Norway', 'Sweden', 'China', 'Canada'\
     ,'Tanzania', 'Denmark', 'South_Korea', 'Italy', 'Finland', 'Taiwan','Russia', 'Philippines', 'Slovenia'\
     , 'Ireland', 'Belgium', 'Mexico','Austria', 'India', 'Brazil', 'Benin', 'New_Zealand','United_Arab_Emirates'\
     , 'Ukraine', 'Poland', 'Israel', 'Portugal','Tunisia', 'Mali', 'Slovakia', 'Zimbabwe', 'Thailand', 'Panama'\
     ,'Indonesia', 'Singapore', 'Greece', 'Senegal', 'Nicaragua','Hungary', 'Czech_Republic', 'Macedonia', 'Chile'\
     , 'Uruguay','Malaysia', 'Algeria', 'Nepal', 'Mauritania', 'Croatia']

cmd = '''review_rdd = reviews\
.map(lambda x : (x.id , Row(id = x.id , avg_review = x.rating \
, recent_review = get_recentReviews(x.date), nb_review = 1\
,version = set([x.version])'''
cmd+=",country = set([x.country])"
#for c in list_countries:
#    cmd+=","+c+''' = int( escape(x.country) == "'''+c+'''")'''
for i in range(1,6):
    cmd+=",review_rating_"+str(i)+" = int(x.rating == "+str(i)+")  "
cmd+=")))"
exec cmd

#Group step
cmd = '''review_rdd = review_rdd.reduceByKey(lambda x1 ,x2 : Row(\
 avg_review = x1.avg_review + x2.avg_review\
   ,recent_review = x1.recent_review + x2.recent_review, nb_review = x1.nb_review + x2.nb_review'''
#for c in list_countries:
#    cmd+=" , "+c+" = x1."+c+" + x2."+c
cmd+=", country = x1.country.union(x2.country)"
cmd+=", version = x1.version.union(x2.version)"
for i in range(1,6):
    cmd+=", review_rating_"+str(i)+" = x1.review_rating_"+str(i)+" + x2.review_rating_"+str(i)+" "
cmd+="))"
exec cmd

# Clean the grouped rdd
cmd = '''review_rdd = review_rdd.map(lambda (id , x) : [ id \
,  1.0*x.avg_review /  x.nb_review\
   , x.recent_review,  x.nb_review'''
#for c in list_countries:
#    cmd+=" , 1.0* x."+c+"/ x.nb_review"
cmd+=",  x.country.pop()"
cmd+=",  len(x.version) " # -1 if want number of updates
for i in range(1,6):
    cmd+=", 1.0*x.review_rating_"+str(i)+" / x.nb_review"
cmd+="])"
exec cmd

#Put back into dataframe
grp_reviews = sqlCtx.createDataFrame(review_rdd, ["id","avg_review"\
      ,"recent_reviews","nb_reviews","country","versions"]+["review_rating_"+str(i) for i in range(1,6)])
    #,"recent_reviews","nb_reviews"] + list_countries + ["versions"]+["review_rating_"+str(i) for i in range(1,6)])
    
#Join with predictors 
predictors = predictors.join(grp_reviews,["id"],"left").fillna("no_country",["country"])

In [ ]:
# Generate DL Projection
sqlCtx.registerDataFrameAsTable(predictors, "predictors")
derived_feats = sqlCtx.sql("SELECT id, device\
    ,CAST((7*download_sum+cumulative_downloads_2015_02) AS float ) AS dl_projection \
    ,CAST((1000000.0*num_ratings/(cumulative_downloads_2015_02 + download_sum))AS float )  AS ratings_per_downloads \
    ,CAST((1.0*num_ratings/(days_since_release+60))AS float )  AS ratings_per_day \
    ,CAST((1000000.0*nb_reviews/download_sum)AS float )  AS review_per_downloads \
    ,CAST((1.0*recent_reviews/nb_reviews)AS float )  AS review_recent_over_old \
    ,CAST((1.0*cumulative_downloads_2015_02/(days_since_release+1))AS float )  AS downloads_per_day_before \
    FROM predictors")
predictors = predictors.join(derived_feats,["id","device"],"left")

#we could group by continent or use the market potential
list_countries+=["no_country"]
#for co in list_countries:
#    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
#    predictors=sqlCtx.sql('''SELECT *, CASE WHEN (country = "'''+co+'''") THEN 1 ELSE 0 END AS '''+co+''' FROM predictors''')
def get_market(country,device):
    #http://blog.nelso.com/2010/06/iphone-os-penetration-by-country.html
    if device == "iphone":
        return {
            "United_States" : 10683403,
            "France" : 2248817,
            "Japan" : 1378903,
            "Spain" : 377346,
            "United_Kingdom" : 2551128,
            "Germany" : 1117716,
            "Hong_Kong" : 299720,
            "Switzerland" : 399364,
            "Netherlands" : 372539,
            "Australia" : 1207428,
            "Norway" : 154218,
            "Sweden" : 281622,
            "China" : 725358,
            "Canada" : 919074,
            "Denmark" : 151426,
            "Italy" : 648718,
            "Taiwan" : 174226,
            "Mexico" : 215326,
            "Austria" : 156322,
            "Brazil" : 219339,
            "Poland" : 72114,
            "Singapore" : 402922,
            "Hungary" : 33219,
            "Czech_Republic" : 42753,
            'South_Korea': 530235,
            "Russia" :246421   
    }.get(country, 0) 
    else:
        return{
            "United_States" : 223269,
            "France" : 2724,
            "Japan" : 2293,
            "Spain" : 1494,
            "United_Kingdom" : 4197,
            "Germany" : 3403,
            "Hong_Kong" : 2306,
            "Switzerland" : 1698,
            "Netherlands" : 2554,
            "Australia" : 1400,
            "Norway" : 1333,
            "Sweden" : 1188,
            "China" : 12516,
            "Canada" : 6275,
            "Denmark" : 753,
            "Italy" : 1370,
            "Taiwan" : 1356,
            "Mexico" : 3380,
            "Austria" : 493,
            "Brazil" : 2014,
            "Poland" : 324,
            "Singapore" : 1453,
            "Hungary" : 211,
            "Czech_Republic" : 203,
            'South_Korea': 2416,
            "Russia" :2183
        }.get(country, 0) 
    
    
    return 1
sqlCtx.registerFunction("get_market", get_market,returnType=IntegerType())

sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, get_market(country,device) AS market_size FROM predictors''')

In [ ]:
s

In [ ]:
predictors.cache()

In [ ]:
predictors.count()

In [ ]:
predictors.toPandas()

In [ ]:

predictors = predictors.fillna(0)

In [ ]:
predictors.toPandas().to_csv("predictors_spark.csv", sep='\t',encoding='utf-8')

#### ML  pipeline

In [ ]:
predictor_names = predictors.rdd.top(1)[0].asDict().keys()
predictor_names.remove("name")
predictor_names.remove("id")
predictor_names.remove("category")
predictor_names.remove("device")
predictor_names.remove("country")

predictors_labelPoints = predictors.rdd\
.map(lambda x: ((x.id,x.device) ,x))\
.join(output.rdd.map(lambda x: ((x.id,x.device) ,x.cumulative_downloads_2016_02)))\
.map(lambda x : (x[0] , LabeledPoint(x[1][1], [x[1][0].asDict()[col] for col in predictor_names])))

predictors_labelPoints.cache()
predictors_labelPoints.top(1)

In [ ]:
def get_preformance_old(labelsAndPredictions):
    total = labelsAndPredictions.count()
    pred_threshold = labelsAndPredictions.takeOrdered(int(0.01*total),lambda x:-x.pred)[-1].pred
    true_threshold = labelsAndPredictions.takeOrdered(int(0.01*total),lambda x:-x.true)[-1].true
    pred_top = labelsAndPredictions.filter(lambda x: x.pred >= pred_threshold ).map(lambda x : ((x.id,x.device),1))
    true_top = labelsAndPredictions.filter(lambda x: x.true >= true_threshold ).map(lambda x : ((x.id,x.device),1)) 
    
    if pred_top.count() > total*0.01 : return -1
    
    return 10000.0*pred_top.join(true_top).count()/total


def get_preformance(labelsAndPredictions):
    total = labelsAndPredictions.count()
    pred_top = labelsAndPredictions.takeOrdered(int(0.01*total),lambda x:-x.pred).map(lambda x : ((x.id,x.device),1))
    true_top = labelsAndPredictions.takeOrdered(int(0.01*total),lambda x:-x.true).map(lambda x : ((x.id,x.device),1))
    
    if pred_top.count() > total*0.01 : return -1
    
    return 10000.0*pred_top.join(true_top).count()/total

In [ ]:
K = 5
predictors_cvSplit = predictors_labelPoints.randomSplit([1.0/K] * K)

for fold in range(K):
    test_rdd = predictors_cvSplit[fold]
    train_rdd = sc.union([predictors_cvSplit[i]  for i in range(K) if (i!= fold)])
    
    model = GradientBoostedTrees.trainRegressor(train_rdd.map(lambda x:x[1])\
                                ,categoricalFeaturesInfo={}\
                                                , numIterations=500\
                                               ,maxDepth=1\
                                               ,learningRate = 0.01)
    
    prediction = model.predict(test_rdd.map(lambda x:x[1].features))
    labelsAndPredictions = test_rdd.map(lambda x: x[1].label)\
    .zip(prediction)\
    .zip(test_rdd.map(lambda x : x[0]))\
    .map(lambda (tuple_true_pred , tuple_id) :\
         Row(id = tuple_id[0], device = tuple_id[1] , pred = tuple_true_pred[1],true = tuple_true_pred[0]))
    print get_preformance(labelsAndPredictions)
    #print get_preformance_old(labelsAndPredictions)
    
    

In [ ]:
predictor_names

In [ ]:
   
    

print list(predictors.columns.values)
np.random.seed(1)
kf = KFold(len(predictors), n_folds=K)
new_top = []
for train, test in kf:

    old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12], output.as_matrix()[train,5])
    y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])


    #model
    #mod=linear_model.LinearRegression(fit_intercept=False)\ 
    #mod=linear_model.Lasso(alpha=100,fit_intercept=False)\
    #mod = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')\ 
    #mod= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 100)\
    #mod= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    
    #mod = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    #.fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5]) 

    #y_pred =  mod.predict(predictors.as_matrix()[test,4:])
    new_top.append(metric(y_pred,output.as_matrix()[test,5]))

print 1.0*sum(new_top)/len(new_top)

In [ ]:
predictor_names

In [ ]:
LabeledPoint(1,[2])

In [ ]:
#Use something like this in the classification step
def even(x): return x % 2 == 0
def odd(x): return not even(x)
rdd = sc.parallelize(range(20))

rdd_odd, rdd_even = (rdd.filter(f) for f in (odd, even))

## To Pandas


In [ ]:
predictors_df = predictors

In [ ]:

predictors_pd = pd.read_csv('predictors_spark.csv',sep='\t').drop('Unnamed: 0', 1).fillna(0)


numerical_predictors = list(predictors_pd.columns.values)
numerical_predictors.remove("id");numerical_predictors.remove("device");
numerical_predictors.remove("category");numerical_predictors.remove("name");
numerical_predictors.remove("country");

predictors_id = predictors_pd[["id","device"]].as_matrix()

predictors_matrix = predictors_pd[numerical_predictors].as_matrix()

output_pd = pd.merge(predictors_pd,output.toPandas(),on = ["id","device"],how="left")\
[["id","device", "cumulative_downloads_2016_02"]]
output_matrix = output_pd["cumulative_downloads_2016_02"].as_matrix()

#predictors_matrix

In [ ]:
#This is the metric we use to determine our performance
def metric(y_pred,y_test,percent=1):
    top = int(len(y_pred)/100.0*percent)
    return (len(set([i[0] for i in sorted(enumerate(y_pred), key=lambda x:x[1],reverse=True)][0:top])
       .intersection([i[0] for i in sorted(enumerate(y_test), key=lambda x:x[1],reverse=True)][0:top])
               ))/(percent/100.0)/len(y_pred)*100

In [ ]:
output.head(1)

In [ ]:
old_predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1).fillna(0)\
.rename(columns={'maxStep': 'max_step', 'minStep': 'min_step','std':'downloads_std','Health and Fitness':'Health_and_Fitness'\
                ,'versions':'num_versions' })

In [ ]:
old_list = list(old_predictors.columns.values)
new_list = (predictors_pd.columns.values)
print old_list

In [ ]:
for n in new_list:
    try:
        print str(n) + " : "+str((sum(np.absolute(predictors_pd.sort_values(["id","device"])[n].as_matrix()\
                                                 -old_predictors.sort_values(["id","device"])[n].as_matrix())))\
                                /sum(np.absolute(predictors_pd.sort_values(["id","device"])[n].as_matrix())))
    except:
        print "error on: "+str(n)
            

In [ ]:

K = 5
print list(predictors_pd.columns.values)
np.random.seed(1)
kf = KFold(len(predictors_pd), n_folds=K)
new_top = []
for train, test in kf:

    #old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12].astype(float), output['cumulative_downloads_2016-02'].as_matrix()[train])
    #y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])


    #model
    #mod=linear_model.LinearRegression(fit_intercept=False)\ 
    #mod=linear_model.Lasso(alpha=100,fit_intercept=False)\
    #mod = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')\ 
    #mod= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 100)\
    #mod= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
    
    mod = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
    .fit(predictors_matrix[train].astype(float), np.log(output_matrix[train]) )

    y_pred = mod.predict(predictors_matrix[test].astype(float))
    err = (metric(y_pred,output_matrix[test]))
    new_top.append(err)
    print err

print 1.0*sum(new_top)/len(new_top)

In [ ]:
plt.scatter(predictors_pd[predictors_pd.days_since_release != 0].sort_values(["id","device"])["id"].as_matrix()\
     ,predictors_pd[predictors_pd.days_since_release != 0].sort_values(["id","device"])["days_since_release"].as_matrix()
           )

In [ ]:
temp = 3

train_days = np.power(\
                      np.matlib.repmat(predictors_pd[predictors_pd.days_since_release != 0]\
                              .sort_values(["id","device"])[["id"]].as_matrix()*1.0/100000000,1,temp+1),np.array(range(temp+1)))

test_days = predictors_pd[predictors_pd.days_since_release != 0]\
                              .sort_values(["id","device"])[["days_since_release"]].as_matrix()
    
day_mod=linear_model.LinearRegression(fit_intercept=False).fit(train_days,test_days )

plt.scatter(np.concatenate((train_days[:,1],train_days[:,1])), \
        np.concatenate((test_days[:,0],np.sum(np.dot(train_days,\
          np.diag(day_mod.coef_[0,:])),axis=1))),\
       c=np.concatenate((np.ones(len(train_days)),2*np.ones(len(train_days)))),\
       alpha=.1)

print day_mod.coef_

In [ ]:
day_mod=linear_model.LinearRegression(fit_intercept=False).fit(train_days,test_days )

In [ ]:
day_mod.coef_


In [ ]:
plt.scatter(np.concatenate((train_days[:,1],train_days[:,1])), \
            np.concatenate((test_days[:,0],np.sum(np.dot(train_days,\
              np.diag(day_mod.coef_[0,:])),axis=1))),\
           c=np.concatenate((np.ones(len(train_days)),2*np.ones(len(train_days)))),\
           alpha=.1)

In [ ]:
np.sum(np.dot(train_days,np.diag([  1.73879033e-24,  -8.16149354e-06,   4.19123763e-15,\
         -2.24536991e-16,  -2.24586827e-16])),axis=1)

In [ ]:
np.concatenate((test_days[:,0],np.sum(np.dot(train_days,\
              np.diag([  1.73879033e-24,  -8.16149354e-06,   4.19123763e-15,\
         -2.24536991e-16,  -2.24586827e-16])),axis=1))) 

In [ ]:
(np.ones(len(train_days)),2*np.ones(len(train_days)))

In [ ]:
np.transpose(test_days)

In [ ]:
np.concatenate((train_days[:,1],train_days[:,1]))

In [ ]:
train_days[0,:]

In [ ]:
train_days = np.power(\
                      np.matlib.repmat(predictors_pd[predictors_pd.days_since_release != 0]\
                              .sort_values(["id","device"])[["id"]].as_matrix()*1.0/100000000,1,temp+1),np.array(range(temp+1)))

In [ ]:
281704574**3

In [ ]:
predictors_pd.sort_values(["id","device"])[["nb_reviews","cumulative_downloads_2015_02","review_per_downloads","review_per_day"]]

In [ ]:
old_predictors.sort_values(["id","device"])[["num_review","cumulative_downloads_2015-02","review_per_downloads"]]

In [ ]:
predictors_pd

In [ ]:
mem = pd.read_csv('../iphones_per_country.csv').fillna(0)


In [ ]:
int(mem[mem.Country == "Mexico"]["iPhone"])

In [ ]:
mem["Country"]

In [ ]:
int(mem[mem.Country == "Mexico"]["iPhone"])

In [ ]:
s=[]
for l in list_countries:
    try:
        print '''"'''+l+'''" : '''+str(int(mem[mem.Country == l.replace("_"," ")]["iPad"]))+","
    except:
        s.append(l)
        t+=1
print s
